In [2]:
import numpy as np
import pandas as pd

from auxiliary import values as v
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from nuclei_segmentation.quality_control import metrics, visualization

In [3]:
v.data_path = '/mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/'

pred_paths = v.data_path + 'Gr1/Segmentation/Nuclei/QC_CROP/'
gt_path = (
        v.data_path 
        + 'Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_GT.nii.gz'
)

gt = imaging.read_image(gt_path, axes='XYZ')

In [4]:
from itertools import permutations, chain


# Cellpose params
steps_type = ['2D', '3D']
steps_thr = list(np.round(np.linspace(.2, .8, 8), 1))
steps_prob_thr = list(np.round(np.linspace(0, .5, 5), 1))

# Preprocessing steps
normalization_methods = ['norm_minmax', 'norm_percentile']
pre_steps_top = ['isotropy']
pre_steps_bottom = [
    'bilateral', 
    'anisodiff'
]

pre_steps_permutations = list(chain(*[
    permutations(pre_steps_bottom, r) 
    for r in range(1, len(pre_steps_bottom) + 1)
]))

# Postprocessing steps
post_steps = [
    # 'remove_small_objects',
    '3d_connected_component_analysis',
    'merge_by_volume',
    'clean_boundaries_morphology',
]

post_steps_permutations = list(chain(*[
    permutations(post_steps, r) 
    for r in range(1, len(post_steps) + 1)
])) + [()]

pipelines_dict = {}

# Create pipelines for each type (2D/3D)
for step in steps_type:
    # Apply threshold only for 2D
    if step == '2D':
        for thr in steps_thr:
            for norm_method in normalization_methods:
                for i, pre_steps in enumerate(pre_steps_permutations):
                    for j, post_steps in enumerate(post_steps_permutations):
                        pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                        pipeline = pipeline_pre + list(post_steps)
                        
                        # Include threshold for 2D pipelines
                        name = f"pipeline_{step}_thr_{thr}_{norm_method}_{i}_{j}"
                        pipelines_dict[name] = {
                            'pipeline': pipeline,
                            'type': step,
                            'threshold': thr
                        }
    else:  # For 3D pipelines, ignore thresholds
        for norm_method in normalization_methods:
            for prob_t in steps_prob_thr:
                for i, pre_steps in enumerate(pre_steps_permutations):
                    for j, post_steps in enumerate(post_steps_permutations):
                        pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                        pipeline = pipeline_pre + list(post_steps)
                        
                        # No threshold for 3D pipelines
                        name = f"pipeline_{step}_prob{prob_t}_{norm_method}_{i}_{j}"
                        pipelines_dict[name] = {
                            'pipeline': pipeline,
                            'type': step,
                            'prob_threshold': prob_t
                        }

print('Total pipelines:', len(pipelines_dict))

Total pipelines: 896


In [8]:
pipelines_dict

{'pipeline_2D_thr_0.2_norm_minmax_0_0': {'pipeline': ['isotropy',
   'norm_minmax',
   'bilateral',
   'remove_small_objects'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_minmax_0_1': {'pipeline': ['isotropy',
   'norm_minmax',
   'bilateral',
   '3d_connected_component_analysis'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_minmax_0_2': {'pipeline': ['isotropy',
   'norm_minmax',
   'bilateral',
   'watershed'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_minmax_0_3': {'pipeline': ['isotropy',
   'norm_minmax',
   'bilateral',
   'remove_small_objects',
   '3d_connected_component_analysis'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_minmax_0_4': {'pipeline': ['isotropy',
   'norm_minmax',
   'bilateral',
   'remove_small_objects',
   'watershed'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_minmax_0_5': {'pipeline': ['isotropy',
   'norm_minmax',
   'bilateral',
   '3d_connected_comp

In [5]:
bar = LoadingBar(len(pipelines_dict))

results = []
for name, config in pipelines_dict.items():
    print(f"\nPipeline: {name}")
    
    file_name = f'20190208_E2_DAPI_decon_0.5_crop_{name}.nii.gz'
    
    out_path = pred_paths + 'Stats/' + file_name.replace('.nii.gz', '.tif')
    pred = imaging.read_image(pred_paths + file_name, axes='XYZ')
    
    dice, _ = metrics.dice_coef(pred, gt, thr_overlap=.6)
    jaccard = metrics.volume_jaccard_index(pred, gt)
    res, stats = metrics.segmentation_stats(pred, gt, thr_overlap=.6)

    results.append({
        'test_name': name,
        'dice': dice,
        'jaccard': jaccard,
        '#cells': stats['total'],
        'correct': stats['correct'],
        'missing': stats['missing'],
        'over_segmented': stats['over_segmented'],
        'under_segmented': stats['under_segmented'],
        'confused': stats['confused']
    })
    
    visualization.save_comparison(pred, res, out_path)
    bar.update()
    
bar.end()


Pipeline: pipeline_2D_thr_0.2_norm_minmax_0_0
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 135 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 135 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 135 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
[                                                  ] 0.11%
Pipeline: pipeline_2D_thr_0.2_norm_minmax_0_1
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 148 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 148 labels...
INFO     [timagetk.c

In [9]:
results_df = pd.DataFrame(results)
results_df.to_csv(pred_paths + 'Stats/results.csv', index=False)

In [7]:
results_df

,test_name,dice,jaccard,#cells,correct,missing,over_segmented,under_segmented,confused
0,pipeline_2D_thr_0.2_norm_minmax_0_0,0.471286,0.499171,101.0,55.45,13.04,1.98,39.60,2.97
1,pipeline_2D_thr_0.2_norm_minmax_0_1,0.521624,0.509619,143.0,48.95,2.69,9.09,19.58,22.38
2,pipeline_2D_thr_0.2_norm_minmax_0_2,0.482135,0.581432,264.0,36.36,2.70,58.33,2.65,2.65
3,pipeline_2D_thr_0.2_norm_minmax_0_3,0.471286,0.499171,101.0,55.45,13.04,1.98,39.60,2.97
4,pipeline_2D_thr_0.2_norm_minmax_0_4,0.444942,0.572361,238.0,31.51,12.97,62.18,3.36,2.94
...,...,...,...,...,...,...,...,...,...
891,pipeline_3D_norm_percentile_3_11,0.465955,0.584389,239.0,33.47,10.22,61.09,5.44,0.00
892,pipeline_3D_norm_percentile_3_12,0.438200,0.572945,176.0,51.14,17.93,41.48,7.39,0.00
893,pipeline_3D_norm_percentile_3_13,0.438200,0.572945,176.0,51.14,17.93,41.48,7.39,0.00
894,pipeline_3D_norm_percentile_3_14,0.438200,0.572945,176.0,51.14,17.93,41.48,7.39,0.00
